# Import database for yield rate at year .... 

In [ ]:
import pandas as pd
import re
%matplotlib inline

DATA_FOLDER = '~/data/'

TRADPRICE = DATA_FOLDER+"tradprice2019.csv"

# Clean Database

## Remove useless columns and rows with unreadable data

In [ ]:
tradprice = pd.read_csv(TRADPRICE)
tradprice.head()

#Remove columns that are not needed
# Columns with only NaN values
tradprice = tradprice.dropna(axis=1, how='all')
# Columns with only 0 values
tradprice = tradprice.loc[:, (tradprice != 0).any(axis=0)]
# Columns Period, Geo Level, State ANSI
tradprice = tradprice.drop(columns=['Year','Program','Period', 'Geo Level', 'State ANSI', 'Domain', 'Domain Category'])

#state how many rows with (D) value have been removed
print("Number of rows with (D) value: ", len(tradprice[tradprice['Value'].str.contains(r'\(D\)')]))
#remove rows where value colummn = (D) with or without spaces
tradprice = tradprice[~tradprice['Value'].str.contains(r'\(D\)')]

#state how many rows with (X) value have been removed
print("Number of rows with (X) value: ", len(tradprice[tradprice['Value'].str.contains(r'\(X\)')]))
#remove rows where value colummn = (X) with or without spaces
tradprice = tradprice[~tradprice['Value'].str.contains(r'\(X\)')]

#state how many rows with (NA) value have been removed
print("Number of rows with (NA) value: ", len(tradprice[tradprice['Value'].str.contains(r'\(NA\)')]))
#remove rows where value colummn = (NA) with or without spaces
tradprice = tradprice[~tradprice['Value'].str.contains(r'\(NA\)')]

#State how many rows with MAPLE SYRUP have been removed
print("Number of rows with MAPLE SYRUP: ", len(tradprice[tradprice['Commodity'].str.contains(r'MAPLE SYRUP')]))
#remove rows where commodity contains MAPLE SYRUP 
tradprice = tradprice[~tradprice['Commodity'].str.contains(r'MAPLE SYRUP')]



tradprice.head(7)

## Unit conversion in $/short ton

In [ ]:
# Function to extract and clean the unit
def extract_clean_unit(data_item):
    match = re.search(r'IN\s+\$(\s*/\s*[A-Z]+)', data_item)  # Look for "IN" followed by the unit
    if match: 
        return f"${match.group(1).strip().replace(' ', '')}"  # Clean spaces to get "BU/ACRE"
    return None  # Return None if no match is found

# Applying the function to create a new 'Unit' column
tradprice['Unit'] = tradprice['Data Item'].apply(extract_clean_unit)

# Display the updated DataFrame
tradprice.head(7)

#print unique values of 'Unit' column
print(tradprice['Unit'].unique())

In [ ]:
#print column with 'Unit' = '$/BARREL'
print(tradprice[tradprice['Unit'] == '$/BARREL'])
#print the number of rows with 'Unit' = '$/BARREL'
print(len(tradprice[tradprice['Unit'] == '$/BARREL']))

In [ ]:
#print column with 'Unit' = '$/BOX'
print(tradprice[tradprice['Unit'] == '$/BOX'])
#print the number of rows with 'Unit' = '$/BOX'
print(len(tradprice[tradprice['Unit'] == '$/BOX']))

In [ ]:
tradprice['Value'] = tradprice['Value'].str.replace(',', '', regex=False)  # Remove thousands separator before conversion
# Convert 'Value' column to numeric, forcing errors to NaN
tradprice['Value'] = pd.to_numeric(tradprice['Value'], errors='coerce')

# Define conversion ratios
conversion_ratios = {
    '$/TON': 1,  # Reference value with Short tons and not metric tons used here
    '$/LB': 0.0005,
    '$/CWT': 0.05, #since 1 CWT = 100 lb
    '$/BU (CORN/SORGHUM/RYE/FLAXSEED)': 0.028,  #source: TABLES 5/6 https://www.ers.usda.gov/webdocs/publications/41880/33132_ah697_002.pdf 
    '$/BU (BARLEY)': 0.024, #source: same
    '$/BU (WHEAT/SOYBEANS)': 0.03, #source: same
    '$/BU (OATS)': 32*0.0005, #source: same
    '$/BARREL': 100*0.0005, #since 1 barrel of cranberry = 100lb of fruits source: same
    '$/BOX (LEMONS)': 38*0.0005,  #since 1 box of lemons = 38 lb source: same
    '$/BOX (ORANGES,FL)': 43*0.0005,  #since 1 box of oranges = 43 lb source: same
    '$/BOX (ORANGES,TX)': 42*0.0005,  #since 1 box of oranges = 42 lb source: same
    '$/BOX (ORANGE, AZ/CA)': 38*0.0005,  #since 1 box of oranges = 38 lb source: same
    '$/BOX (GRAPEFRUIT, FL/TX)': 40*0.0005,  #since 1 box of oranges = 40 lb source: same
    '$/BOX (GRAPEFRUIT, AZ/CA)': 34*0.0005,  #since 1 box of oranges = 34 lb source: same
    '$/BOX (TANGERINES, AZ/CA)': 25*0.0005,  #since 1 box of oranges = 25 lb source: same
    '$/BOX (TANGERINES, FL)': 43*0.0005,  #since 1 box of oranges = 43 lb source: same
    }


# Function to convert values to tons/acre
def convert_to_tons_acre(row):
    value = row['Value']
    unit = row['Unit']

    if pd.isna(value):  # Check if the value is NaN
        return None
    if unit == '$/TON':
        return value  # No conversion needed
    elif '$/BU' in unit:
        if 'CORN' in row['Commodity'] or 'SORGHUM' or 'RYE' in row['Commodity'] or 'FLAXSEED' in row['Commodity']:
            return value / conversion_ratios['$/BU (CORN/SORGHUM/RYE/FLAXSEED)']
        elif 'BARLEY' in row['Commodity']:
            return value / conversion_ratios['$/BU (BARLEY)']
        elif 'WHEAT' in row['Commodity'] or 'SOYBEANS' in row['Commodity']:
            return value / conversion_ratios['$/BU (WHEAT/SOYBEANS)']
        elif 'OATS' in row['Commodity']:
            return value / conversion_ratios['$/BU (OATS)']
    elif unit == '$/LB':
        return value / conversion_ratios['$/LB']
    elif unit == '$/CWT':
        return value / conversion_ratios['$/CWT']
    elif unit == '$/BARREL':
        return value / conversion_ratios['$/BARREL']
    elif unit == '$/BOX':
        if 'LEMONS' in row['Commodity']:
            return value / conversion_ratios['$/BOX (LEMONS)']
        elif 'ORANGE' in row['Commodity']:
            if 'FLORIDA' in row['State']:
                return value / conversion_ratios['$/BOX (ORANGES,FL)']
            elif 'TEXAS' in row['State']:
                return value / conversion_ratios['$/BOX (ORANGES,TX)']
            elif 'ARIZONA' in row['State'] or 'CALIFORNIA' in row['State']:
                return value / conversion_ratios['$/BOX (ORANGE, AZ/CA)']
        elif 'GRAPEFRUIT' in row['Commodity']:
            if 'FLORIDA' in row['State'] or 'TEXAS' in row['State']:
                return value / conversion_ratios['$/BOX (GRAPEFRUIT, FL/TX)']
            elif 'ARIZONA' in row['State'] or 'CALIFORNIA' in row['State']:
                return value / conversion_ratios['$/BOX (GRAPEFRUIT, AZ/CA)']
        elif 'TANGERINES' in row['Commodity']:
            if 'FLORIDA' in row['State']:
                return value / conversion_ratios['$/BOX (TANGERINES, FL)']
            elif 'ARIZONA' in row['State'] or 'CALIFORNIA' in row['State']:
                return value / conversion_ratios['$/BOX (TANGERINES, AZ/CA)']
        
    return None  # Return None for unrecognized units

# Applying the conversion function to create the 'new_values' column
tradprice['new_values'] = tradprice.apply(convert_to_tons_acre, axis=1)

# Dropping the old 'Value' and 'Unit' columns
tradprice.drop(columns=['Value', 'Unit'], inplace=True)

# Renaming 'new_values' to 'price' and adding unit column
tradprice.rename(columns={'new_values': 'price'}, inplace=True)
tradprice['Unit'] = '$/TON'  # Add a new unit column with 'TON/ACRE'


tradprice.head(7)

## Prepare data to pivot table

In [ ]:
#Check if some rows have NaN price and print them to see the unit (if it is not TON/ACRE or one of the other units already implemented)
missing_values = len(tradprice[tradprice['price'].isnull()])
print("Number of rows with NaN price: ", missing_values)
#print rows with NaN price
if missing_values > 0:
    print(tradprice[tradprice['price'].isnull()])

#Check if some rows have negative price and print how many of them
negative_values = len(tradprice[tradprice['price'] < 0])
print("Number of rows with negative price: ", negative_values)
#Remove rows with negative price
tradprice = tradprice[tradprice['price'] >= 0]

In [ ]:
# Function to extract the commodity name from the Data Item
def extract_commodity(data_item):
    match = re.search(r'^(.*?)\s*-\s*', data_item)
    if match:
        return match.group(1).strip()  # Return the part before the dash
    return None

# Updating the Commodity column with extracted values
tradprice['Commodity'] = tradprice['Data Item'].apply(extract_commodity)

tradprice.head(200)


In [ ]:
# Create a new column 'types' based on the commodity content
def categorize_commodity(commodity):
    if 'FRESH MARKET' in commodity:
        return 'FRESH MARKET'
    elif 'PROCESSING' in commodity:
        return 'PROCESSING'
    else:
        return 'GENERAL'

# Apply the function to the Commodity column
tradprice['types'] = tradprice['Commodity'].apply(categorize_commodity)

# Remove "PROCESSING" and "FRESH MARKET" from the 'Commodity' column, regardless of format
tradprice['Commodity'] = tradprice['Commodity'].replace({
    r'PROCESSING': '', 
    r'FRESH MARKET': ''
}, regex=True).str.strip()

# Remove any trailing commas or extra spaces left behind
tradprice['Commodity'] = tradprice['Commodity'].str.replace(r',\s*$', '', regex=True).str.strip()

#print how many rows have the word 'UTILIZED' or 'IN SHELL' or 'IMPORVED' in the 'Commodity' commas
print("Number of rows with UTILIZED in Commodity: ", len(tradprice[tradprice['Commodity'].str.contains(r'UTILIZED')]))
print("Number of rows with IN SHELL in Commodity: ", len(tradprice[tradprice['Commodity'].str.contains(r'IN SHELL')]))
print("Number of rows with IMPROVED in Commodity: ", len(tradprice[tradprice['Commodity'].str.contains(r'IMPROVED')]))


# Remove the word 'UTILIZED' and 'IN SHELL' and 'IMPROVED' if present in the 'Commodity' column even if there are words, colon before or after
tradprice['Commodity'] = tradprice['Commodity'].str.replace(r'\s*UTILIZED\s*', '', regex=True).str.strip()
# Remove any trailing commas or extra spaces left behind
tradprice['Commodity'] = tradprice['Commodity'].str.replace(r',\s*$', '', regex=True).str.strip()
tradprice['Commodity'] = tradprice['Commodity'].str.replace(r'\s*IN SHELL\s*', '', regex=True).str.strip()
# Remove any trailing commas or extra spaces left behind
tradprice['Commodity'] = tradprice['Commodity'].str.replace(r',\s*$', '', regex=True).str.strip()
tradprice['Commodity'] = tradprice['Commodity'].str.replace(r'\s*IMPROVED\s*', '', regex=True).str.strip()
# Remove any trailing commas or extra spaces left behind
tradprice['Commodity'] = tradprice['Commodity'].str.replace(r',\s*$', '', regex=True).str.strip()

tradprice.head(200)

In [ ]:
# Step 1: Check for rows with the same Commodity and State but different types
duplicate_rows = tradprice[tradprice.duplicated(subset=['Commodity', 'State'], keep=False) & tradprice['types'].duplicated()]

# Step 2: Calculate the average price for rows with the same Commodity and State but different types
tradprice = tradprice.groupby(['State', 'Commodity'], as_index=False).agg({'price': 'mean', 'types': lambda x: ', '.join(sorted(set(x)))})

# Step 3: Check if duplicates still exist after averaging
duplicates_remaining = tradprice[tradprice.duplicated(subset=['Commodity', 'State'], keep=False)]

# Display the results
print("Duplicates found before averaging:")
print(duplicate_rows)

print("\nAveraged DataFrame:")
print(tradprice)

print("\nDuplicates remaining after averaging:")
print(duplicates_remaining)



## Pivot table 
#### (for better/coherent extraction between diffferent tables later)

In [ ]:
# Now pivoting the DataFrame
tradprice_pivot = tradprice.pivot(index='State', columns='Commodity', values='price')

# Resetting the index to turn the pivot table back into a DataFrame
tradprice_pivot.reset_index(inplace=True)




# Export cleaned and pivoted databases

In [ ]:
# Save the cleaned data to a new CSV file
tradprice_pivot.to_csv(DATA_FOLDER+'tradprice_pivotcleaned.csv', index=False)
tradprice.to_csv(DATA_FOLDER+'tradprice_cleaned.csv', index=False)


